In [15]:
import csv
import pandas as pd
from tqdm import tqdm
from requests.compat import urljoin
import requests
import glob
import os
from bs4 import BeautifulSoup

In [17]:
# !pip install lxml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 1.6 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
# Open types.txt and read the types
with open('../data/gold_standard_corpus.tsv', 'r') as f:
    pmcids = f.read().splitlines()

In [27]:
pmcids

['PMC4792959',
 'PMC4556948',
 'PMC5993813',
 'PMC3174205',
 'PMC5962829',
 'PMC3874094',
 'PMC3792120',
 'PMC4901335',
 'PMC3581133',
 'PMC2935479',
 'PMC5225553',
 'PMC5744400',
 'PMC3281816',
 'PMC3583137',
 'PMC4022742',
 'PMC3542345',
 'PMC4452330',
 'PMC4464872',
 'PMC4872455',
 'PMC3651197',
 'PMC3362782',
 'PMC5817132',
 'PMC4313693',
 'PMC4489904',
 'PMC4552872',
 'PMC5376652',
 'PMC5070310',
 'PMC5921292',
 'PMC5641157',
 'PMC3751948',
 'PMC5472290',
 'PMC4649626',
 'PMC5502978',
 'PMC4767726',
 'PMC3897916',
 'PMC5087830',
 'PMC3585192',
 'PMC5484670',
 'PMC5259676',
 'PMC3024232',
 'PMC3097211',
 'PMC5317055',
 'PMC3648400',
 'PMC5750880',
 'PMC5100220',
 'PMC4749753',
 'PMC5344356',
 'PMC5110973',
 'PMC5708618',
 'PMC3598673',
 'PMC3751959',
 'PMC5131611',
 'PMC5891595',
 'PMC3950279',
 'PMC5972578',
 'PMC5082793',
 'PMC5487420',
 'PMC5106849',
 'PMC3858553',
 'PMC3613406',
 'PMC4167147',
 'PMC3599585',
 'PMC2761781',
 'PMC3899050',
 'PMC2481430',
 'PMC5006041',
 'PMC47908

In [28]:
def get_Json_through_PMCID(pmcid):
    base_url = "https://www.ebi.ac.uk/europepmc/annotations_api/"
    article_url = urljoin(base_url,
                          "annotationsByArticleIds?articleIds=PMC%3A" + pmcid + "&provider=Europe%20PMC&format=JSON")
    r = requests.get(article_url)

    if r.status_code == 200:
        return r
    else:
        return False


In [32]:
def get_epmc_annotations_to_file(PMCids):
    with open('../data/annotations_api.csv', 'w', newline='\n') as f1:
        test_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

        # count = 0
        for each_id in tqdm(PMCids):
            # count = count+1
            # print(each_test_pmc_id + '\t' + str(count))
            json_annotations = get_Json_through_PMCID(each_id[3:])  # Just the number is needed. So remove the PMC from the front
            if json_annotations:
                json_results = json_annotations.json()
                try:
                    pmc_id = json_results[0]['pmcid']
                    # print(pmc_id)
                    for each_annotation in json_results[0]['annotations']:
                        exact = each_annotation['prefix'] + each_annotation['exact'] + each_annotation['postfix']
                        token = each_annotation['tags'][0]['name']
                        ner = each_annotation['type']
                        row = [pmc_id, exact, token, ner]
                        test_writer.writerow(row)
                except(IndexError):
                    print('no annotations found!! '+str(each_id))
            else:
                print('no annotations! '+str(each_id))
                continue


In [33]:
get_epmc_annotations_to_file(pmcids)

100%|█████████████████████████████████████████| 300/300 [00:52<00:00,  5.76it/s]


In [2]:
def extract_sentences_from_sentencised_xml(path_to_PMCids):
    all_files = sorted(glob.glob(path_to_PMCids + '*.xml'))

    with open('../data/full_sentences_from_xmls.csv', 'w', newline='\n') as f1:
        test_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

        # count = 0
        for each_test_pmc_id in tqdm(all_files):
            # count = count+1
            pmc_id = os.path.basename(each_test_pmc_id)[:-4]
            # print(pmc_id + '\t' + str(count))

            with open(each_test_pmc_id, 'r') as f:
                contents = f.read()
                soup = BeautifulSoup(contents, 'lxml')
                for each_sent in soup.find_all('plain'):
                    try:
                        section_type = each_sent.findParents('sectag')[0]['type']
                    except:
                        section_type = 'UNK'
                    if section_type not in ['REF', 'ACK_FUND']:
                        row = [pmc_id, section_type, each_sent.text]
                        test_writer.writerow(row)


                        
                        

In [18]:
extract_sentences_from_sentencised_xml('../data/300_articles_source_files/')

  0%|                                                   | 0/300 [00:00<?, ?it/s]


FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

In [ ]:
def convert_partial_sentences_to_full(path_to_csv_annotationAPI_sentences, path_to_csv_full_sentences,PMC_ids, result_path):
    # count = 0

    annotation_sent_csv = pd.read_csv(path_to_csv_annotationAPI_sentences, names=['pmc_id', 'sentence', 'token', 'ner'],
                                      sep='\t')
    full_sent_csv = pd.read_csv(path_to_csv_full_sentences, names=['pmc_id', 'section', 'sentence'],
                                sep='\t')


    for each_pmc_id in tqdm(PMC_ids):
        # count = count + 1
        # print(each_pmc_id + '\t' + str(count))

        pmc_id_annotation_sents = annotation_sent_csv[annotation_sent_csv['pmc_id'] == each_pmc_id]
        pmc_id_full_sentences = full_sent_csv[full_sent_csv['pmc_id'] == each_pmc_id]
        pmc_id_full_sentences_list = pmc_id_full_sentences['sentence'].tolist()

        full_sentences_match = []

        for iter_a, rows_a in pmc_id_annotation_sents.iterrows():
            # try:
            #     res = [x for x in pmc_id_full_sentences_list if re.search(re.escape(rows_a['sentence']), x)]
            #     if not res[0] in full_sentences_match:
            #         full_sentences_match.append(res[0])
            #     else:
            #         full_sentences_match.append(res[1])
            # except:
            #     full_sentences_match.append('None')
            try:
                res = [x for x in pmc_id_full_sentences_list if re.search(re.escape(rows_a['sentence']), x)]
                full_sentences_match.append(res[0])
            except:
                full_sentences_match.append('None')

        if full_sentences_match:
            pmc_id_annotation_sents = pmc_id_annotation_sents.assign(full_sentence=full_sentences_match)
            pmc_id_annotation_sents['combined'] = pmc_id_annotation_sents.apply(lambda x: list([x['sentence'], x['token'], x['ner']]),
                                                                                        axis=1)
            sent_tags = pmc_id_annotation_sents.groupby('full_sentence')['combined'].apply(list).reset_index(name='tags')
            pmc_id_full_sentences.rename(columns={'sentence': 'full_sentence'}, inplace=True)
            epmc_full_sentence_tags = pd.merge(pmc_id_full_sentences, sent_tags, on='full_sentence', how='left')

            epmc_full_sentence_tags.to_csv(result_path+'Europe_PMC_annotation.csv', mode='a', header=False, sep='\t', index=False)
        else:
            pmc_id_full_sentences.to_csv(result_path+'Europe_PMC_annotation.csv', mode='a', header=False, sep='\t', index=False)
